# Experiment 1, Model 1

#### Model Setup

Run models in the following order, using their output labels as features for the next model:

1. Multilabel Linguistic Classifier
2. Multiclass Person Name + Occupation Sequence Classifier
3. Multilabel Document Classifier

Train the first model and then run it over the entire dataset.

***

* Supervised learning
    * Train, Validate, and (Blind) Test Data: under directory `../data/token_clf_data/experiment_input/`
    * Prediction Data: Data: under directory `../data/token_clf_data/model_output/experiment1/`
* Word Embeddings
    * Custom fastText (word2vec with subwords) embeddings of 100 dimensions trained on the CRC Archives catalog's descriptive metadata (harvested October 2020)
    
***

**Table of Contents**

[1.](#1) Train the Linguistic Classifier

[2.](#2) Predict Over All Data

Load programming resources:

In [1]:
# For custom functions and variables
import utils, utils1, config

# For data analysis
import pandas as pd
import numpy as np
import os, re

# For creating directories
from pathlib import Path

# For preprocessing
from gensim.models import FastText
from gensim import utils as gensim_utils

# For multilabel token classification
import sklearn.metrics
from sklearn.preprocessing import MultiLabelBinarizer
from skmultilearn.problem_transform import ClassifierChain
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier

# For multiclass sequence classification
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

Define resources for the models:

In [2]:
Path(config.experiment_input_path).mkdir(parents=True, exist_ok=True)    # For train, devtest, and blind test data
Path(config.experiment1_output_path).mkdir(parents=True, exist_ok=True)  # For predictions
Path(config.experiment1_agmt_path).mkdir(parents=True, exist_ok=True)    # For agreement metrics

In [3]:
# Model 1:
ling_label_subset = ["B-Generalization", "I-Generalization", "B-Gendered-Role", "I-Gendered-Role", "B-Gendered-Pronoun", "I-Gendered-Pronoun"]
# Model 2:
pers_o_label_subset = ["B-Unknown", "I-Unknown", "B-Feminine", "I-Feminine", "B-Masculine", "I-Masculine", "B-Occupation", "I-Occupation"]
# Model 3:
so_label_subset = ["B-Stereotype", "I-Stereotype", "B-Omission", "I-Omission"]

In [4]:
ling_label_tags = {
    "Gendered-Pronoun": ["B-Gendered-Pronoun", "I-Gendered-Pronoun"], "Gendered-Role": ["B-Gendered-Role", "I-Gendered-Role"],"Generalization": ["B-Generalization", "I-Generalization"]
    }
pers_o_label_tags = {
    "Unknown": ["B-Unknown", "I-Unknown"], "Feminine": ["B-Feminine", "I-Feminine"], "Masculine": ["B-Masculine", "I-Masculine"],
     "Occupation": ["B-Occupation", "I-Occupation"]
    }
so_label_tags = {
    "Stereotype": ["B-Stereotype", "I-Stereotype"], "Omission": ["B-Omission", "I-Omission"]
             }

In [5]:
d = 100  # dimensions of word embeddings (should match utils1.py)

<a id="1"></a>
## 1. Train the Linguistic Classifier

Run a multilabel classifier on the train set of the data, focusing only on applying the Linguistic category of labels: Gendered Pronoun, Gendered Role, and Generalization.

Use a Classifier Chain with Random Forest, as this was the highest-performing multilabel model setup from previous algorithm experiments for the Linguistic labels.

For this experiment, we'll train the model on 40% of the data, rather than 60%.  We'll use fastText embeddings of 100 dimensions, as was used in the model that achieved the above scores.

#### Preprocessing

In [6]:
train_df = pd.read_csv(config.tokc_path+"experiment_input/token_train.csv", index_col=0)
dev_df = pd.read_csv(config.tokc_path+"experiment_input/token_validate.csv", index_col=0)
test_df = pd.read_csv(config.tokc_path+"experiment_input/token_validate.csv", index_col=0)
ling_train = utils1.selectDataForLabels(train_df, "tag", ling_label_subset)
ling_dev = utils1.selectDataForLabels(dev_df, "tag", ling_label_subset)
ling_test = utils1.selectDataForLabels(test_df, "tag", ling_label_subset)

In [8]:
train_data = utils1.loadData(ling_train)
dev_data = utils1.loadData(ling_dev)
test_data = utils1.loadData(ling_test)
print(train_data.shape, dev_data.shape, test_data.shape)
train_data.head()

(298617, 5) (305924, 5) (305924, 5)


,sentence_id,token_id,token,pos,tag
0,2,16,Scope,NN,[O]
1,2,17,and,CC,[O]
2,2,18,Contents,NNS,[O]
3,2,19,:,:,[O]
4,2,20,Sermons,NNS,[O]


In [9]:
assert train_data.shape[0] == len(train_data.token_id.unique())
assert dev_data.shape[0] == len(dev_data.token_id.unique())
assert test_data.shape[0] == len(test_data.token_id.unique())

Combine all the data to run the model over after training:

In [10]:
all_data = pd.concat([train_data, dev_data, test_data])

Create feature matrices:

In [12]:
train_tokens = utils1.zipTokensFeatures(train_data)
dev_tokens = utils1.zipTokensFeatures(dev_data)
all_tokens = utils1.zipTokensFeatures(all_data)
X_train = utils1.makeFastTextFeatureMatrix(train_tokens)
X_dev = utils1.makeFastTextFeatureMatrix(dev_tokens)
X_all = utils1.makeFastTextFeatureMatrix(all_tokens)

Binarize targets:

In [13]:
mlb, y_train = utils1.binarizeTrainTargets(train_data)
y_dev = utils1.binarizeDevTargets(mlb, dev_data)
y_all = utils1.binarizeDevTargets(mlb, all_data)

#### Train & Predict

In [14]:
a = "rf"
clf = ClassifierChain(
    classifier = RandomForestClassifier(random_state=22),
)
clf.fit(X_train, y_train)

ClassifierChain(classifier=RandomForestClassifier(random_state=22),
                require_dense=[True, True])

In [15]:
predictions = clf.predict(X_dev)

#### Evaluate: Strict, All Labels

In [16]:
print("Precision - macro:", sklearn.metrics.precision_score(y_dev, predictions, average="macro", zero_division=0))  # macro = mean of all labels' score
print("Recall - macro:", sklearn.metrics.recall_score(y_dev, predictions, average="macro", zero_division=0))
print("F1 Score - macro:", sklearn.metrics.f1_score(y_dev, predictions, average="macro", zero_division=0))
print("Accuracy - normalized:", sklearn.metrics.accuracy_score(y_dev, predictions, normalize=True))  # fraction of correctly classified samples
print("Accuracy - unnormalized:", sklearn.metrics.accuracy_score(y_dev, predictions, normalize=False))  # number of correctly classified samples

Precision - macro: 0.46238272522632073
Recall - macro: 0.42298863525280694
F1 Score - macro: 0.42251820432443793
Accuracy - normalized: 0.9925798564349315
Accuracy - unnormalized: 303654


In [17]:
print("Total samples:", X_dev.shape[0])

Total samples: 305924


##### Annotation-level Agreement

Join the manual annotations' offsets to the evaluation data:

In [234]:
annot_df = pd.read_csv(config.agg_path+"aggregated_final.csv")#, usecols=["description_id","agg_ann_id", "ann_offsets"])
# Get only the Linguistic annotations
annot_df = annot_df.loc[annot_df.category == "Linguistic"]
annot_df = annot_df[["agg_ann_id", "ann_offsets", "label"]]
annot_df = annot_df.rename(columns={"agg_ann_id":"ann_id"})
# annot_df.head()

In [237]:
dev_token_ids = list(dev_data.token_id.unique())
ling_dev_subset = ling_dev.loc[ling_dev.token_id.isin(dev_token_ids)]

In [238]:
to_add = ling_dev_subset[["ann_id", "token_id", "token_offsets"]]
# Only include annotations with Linguistic labels
to_add = to_add.loc[to_add.ann_id.isin(list(annot_df.ann_id))]
eval_df_joined = eval_df.join(to_add.set_index("token_id"), on="token_id", how="outer")
# Join on the left, as there will be annotations from outside the devtest set in annot_df
print(eval_df_joined.shape)
eval_df_joined = eval_df_joined.join(annot_df.set_index("ann_id"), on="ann_id", how="left")
print(eval_df_joined.shape)  # Looks good!  Same as before join.
eval_df_joined = eval_df_joined.rename(columns={"label":"expected_label"})
eval_df_joined.head()

(3996, 9)
(3996, 11)


,sentence_id,token_id,token,pos,expected_tag,predicted_tag,_merge,ann_id,token_offsets,ann_offsets,expected_label
39.0,5.0,155,his,PRP$,B-Gendered-Pronoun,B-Gendered-Pronoun,true positive,14379,"(913, 916)","(913, 916)",Gendered-Pronoun
41.0,5.0,157,he,PRP,B-Gendered-Pronoun,B-Gendered-Pronoun,true positive,14380,"(928, 930)","(928, 930)",Gendered-Pronoun
62.0,7.0,216,His,PRP$,B-Gendered-Pronoun,B-Gendered-Pronoun,true positive,14382,"(1241, 1244)","(1241, 1244)",Gendered-Pronoun
72.0,7.0,226,he,PRP,B-Gendered-Pronoun,B-Gendered-Pronoun,true positive,14383,"(1315, 1317)","(1315, 1317)",Gendered-Pronoun
218.0,16.0,435,He,PRP,B-Gendered-Pronoun,B-Gendered-Pronoun,true positive,9516,"(677, 679)","(677, 679)",Gendered-Pronoun


Replace the predicted tags with their corresponding labels:

In [239]:
eval_df_joined.expected_label = eval_df_joined.expected_label.fillna("no_label")
eval_df_joined.expected_tag = eval_df_joined.expected_tag.fillna("no_label")
eval_df_joined.predicted_tag = eval_df_joined.predicted_tag.fillna("no_label")
eval_df_joined.predicted_tag.value_counts()

no_label              2234
B-Gendered-Pronoun    1447
B-Gendered-Role        232
B-Generalization        82
I-Generalization         1
Name: predicted_tag, dtype: int64

In [240]:
predicted_labels = list(eval_df_joined.predicted_tag)
predicted_labels = [tag[2:] if tag != "no_label" else tag for tag in predicted_labels]
eval_df_joined.insert(len(eval_df_joined.columns), "predicted_label", predicted_labels)
# eval_df_joined.head()

In [251]:
cols_to_keep = ["sentence_id", "token_id", "token", "expected_label", "predicted_label", "_merge", "token_offsets", "ann_offsets", "ann_id"]
eval_by_ann = utils.implodeDataFrame(eval_df_joined[cols_to_keep], ["ann_id", "ann_offsets"]).reset_index()
exp_labels = list(eval_by_ann["expected_label"])
exp_labels = [labels[0] for labels in exp_labels]
eval_by_ann["expected_label"] = exp_labels
# eval_by_ann.head()

In [248]:
assert eval_by_ann.loc[eval_by_ann.expected_label == "no_label"].shape[0] == 0
assert eval_by_ann.loc[eval_by_ann.expected_label.isna()].shape[0] == 0

Every row should have an annotation label (a Linguistic label in `expected_label`).

In [250]:
ann_agmts = []
token_agmts = (eval_by_ann["_merge"])
for agmts in token_agmts:
    if "true positive" in agmts:
        ann_agmt = "true positive"
    elif "false positive" in agmts:
        ann_agmt = "false positive"
    else:
        ann_agmt = "false negative"
    ann_agmts += [ann_agmt]
assert len(ann_agmts) == eval_by_ann.shape[0]
eval_by_ann.insert(len(eval_by_ann.columns), "annotation_agreement", ann_agmts)
eval_by_ann.head()

,ann_id,ann_offsets,sentence_id,token_id,token,expected_label,predicted_label,_merge,token_offsets,annotation_agreement
0,0,"(1407, 1415)","[5760.0, 5760.0]","[133674, 133674]","[knighted, knighted]",Gendered-Role,"[no_label, Generalization]","[false negative, false positive]","[(1407, 1415), (1407, 1415)]",false positive
1,1,"(9625, 9635)","[nan, nan]","[228678, 228679]","[nan, nan]",Gendered-Role,"[no_label, no_label]","[nan, nan]","[(9625, 9635), (9635, 9636)]",false negative
2,2,"(2426, 2439)","[nan, nan, nan]","[196525, 196526, 196527]","[nan, nan, nan]",Gendered-Role,"[no_label, no_label, no_label]","[nan, nan, nan]","[(2426, 2432), (2433, 2439), (2439, 2440)]",false negative
3,93,"(4141, 4148)","[nan, nan]","[4714, 4715]","[nan, nan]",Gendered-Role,"[no_label, no_label]","[nan, nan]","[(4141, 4148), (4148, 4149)]",false negative
4,1063,"(1532, 1535)",[12112.0],[272117],[his],Gendered-Pronoun,[Gendered-Pronoun],[true positive],"[(1532, 1535)]",true positive


Save the data:

In [254]:
eval_by_ann[["ann_id", "ann_offsets", "token_id", "expected_label", "predicted_label", "annotation_agreement"]].to_csv(
    config.experiment1_agmt_path+"cc-{a}_baseline_fastText{d}_ling_annot_evaluation.csv".format(a=a,d=d)
)

Calculate annotation agreement metrics for each label:

In [292]:
annot_agmt = pd.DataFrame.from_dict({
        "label":[], "false negative":[], "false positive":[],
         "true positive":[], "precision":[], "recall":[], "f1":[]
    })

In [293]:
labels = ling_label_tags.keys()
for label in labels:
    agmt_df = eval_by_ann.loc[eval_by_ann.expected_label == label]
    tp = agmt_df.loc[agmt_df.annotation_agreement == "true positive"].shape[0]
    fp = agmt_df.loc[agmt_df.annotation_agreement == "false positive"].shape[0]
    fn = agmt_df.loc[agmt_df.annotation_agreement == "false negative"].shape[0]
    prec, rec, f1 = utils.precisionRecallF1(tp, fp, fn)
    label_agmt = pd.DataFrame.from_dict({
            "label":[label], "false negative":[fn], "false positive":[fp],
             "true positive":[tp], "precision":[prec], "recall":[rec], "f1":[f1]
        })
    annot_agmt = pd.concat([annot_agmt, label_agmt])
annot_agmt

,label,false negative,false positive,true positive,precision,recall,f1
0,Gendered-Pronoun,131.0,0.0,1401.0,1.000000,0.914491,0.955336
0,Gendered-Role,976.0,4.0,230.0,0.982906,0.190713,0.319444
0,Generalization,408.0,3.0,124.0,0.976378,0.233083,0.376328


Save the scores:

In [262]:
# eval_by_ann.loc[eval_by_ann.expected_label == "Gendered-Role"]["annotation_agreement"].value_counts()     # Looks good
# eval_by_ann.loc[eval_by_ann.expected_label == "Gendered-Pronoun"]["annotation_agreement"].value_counts()  # Looks good
# eval_by_ann.loc[eval_by_ann.expected_label == "Generalization"]["annotation_agreement"].value_counts()    # Looks good
annot_agmt.to_csv(config.experiment1_agmt_path+"cc-{a}_baseline_fastText{d}_ling_annot_agmt.csv".format(a=a,d=d))

##### Loose Agreement

Calculate precision, recall, and F1 score at the token level for each label, where a correct prediction is a prediction with the correct annotation label (not necessarily the correct IOB tag).

Create a copy of the evaluation DataFrame where tags are replaced by label names:

In [7]:
a = "rf"
eval_df = pd.read_csv(config.experiment1_agmt_path+"cc-{a}_ling_baseline_fastText{d}_evaluation.csv".format(a=a,d=d), index_col=0)
loose_eval_df = eval_df.copy()
for label,tags in ling_label_tags.items():
    for tag in tags:
        loose_eval_df["expected_tag"] = loose_eval_df["expected_tag"].replace(to_replace=tag, value=label)
        loose_eval_df["predicted_tag"] = loose_eval_df["predicted_tag"].replace(to_replace=tag, value=label)
# loose_eval_df.head()

In [8]:
loose_eval_df.loc[loose_eval_df.predicted_tag.isna()].shape

(476, 7)

In [9]:
loose_eval_df = loose_eval_df.fillna("O")
loose_eval_df = loose_eval_df.drop(columns=["_merge"])
loose_eval_df = utils.compareExpectedPredicted(loose_eval_df, "_merge", "O")
loose_eval_df.head()

,sentence_id,token_id,token,pos,expected_tag,predicted_tag,_merge
39,5,155,his,PRP$,Gendered-Pronoun,Gendered-Pronoun,true positive
41,5,157,he,PRP,Gendered-Pronoun,Gendered-Pronoun,true positive
62,7,216,His,PRP$,Gendered-Pronoun,Gendered-Pronoun,true positive
72,7,226,he,PRP,Gendered-Pronoun,Gendered-Pronoun,true positive
218,16,435,He,PRP,Gendered-Pronoun,Gendered-Pronoun,true positive


In [12]:
loose_eval_df.to_csv(config.experiment1_agmt_path+"cc-{a}_ling_baseline_fastText{d}_evaluation_loose.csv".format(a=a,d=d))

In [10]:
loose_agmt = pd.DataFrame.from_dict({
        "tag(s)":[], "false negative":[], "false positive":[], "true negative":[], 
         "true positive":[], "precision":[], "recall":[], "f1":[]
    })

In [11]:
for label,tags in ling_label_tags.items():
    labels_agmt_stats = utils.getScoresByTags(loose_eval_df, "_merge", [label])
    loose_agmt = pd.concat([loose_agmt, labels_agmt_stats])
loose_agmt

,tag(s),false negative,false positive,true negative,true positive,precision,recall,f1
0,Gendered-Pronoun,38.0,0.0,NaN,2802.0,1.0,0.986620,0.993265
0,Gendered-Role,229.0,0.0,NaN,438.0,1.0,0.656672,0.792760
0,Generalization,209.0,0.0,NaN,148.0,1.0,0.414566,0.586139


Great!  The performance of this model looks comparable to the model trained on 60% of the data.

Save the data:

In [278]:
loose_agmt.to_csv(config.experiment1_agmt_path+"cc-{a}_baseline_fastText{d}_ling_loose_agmt.csv".format(a=a,d=d))

<a id="2"></a>
## 2. Predict Over All Data

In [19]:
all_predictions = clf.predict(X_all)

In [20]:
pred_df = utils.makePredictionDF(all_predictions, all_data, "tag", "predicted_tag", "O", mlb)
assert pred_df.loc[pred_df.predicted_tag.isna()].shape[0] == 0, "Any NaN values should be replaced with 'O'"
pred_df.head()

,sentence_id,token_id,token,pos,predicted_tag
0,2,16,Scope,NN,O
1,2,17,and,CC,O
2,2,18,Contents,NNS,O
3,2,19,:,:,O
4,2,20,Sermons,NNS,O


In [60]:
print(pred_df.shape[0], len(pred_df.token_id.unique()))

306940 305924


Save the prediction data:

In [21]:
pred_df.to_csv(config.experiment1_output_path+"cc-{a}_ling_baseline_fastText{d}_predictions_ALLDATA.csv".format(a=a,d=d))

#### Evaluate: Strict, All Labels

In [31]:
print("Precision - macro:", sklearn.metrics.precision_score(y_all, all_predictions, average="macro", zero_division=0))  # macro = mean of all labels' score
print("Recall - macro:", sklearn.metrics.recall_score(y_all, all_predictions, average="macro", zero_division=0))
print("F1 Score - macro:", sklearn.metrics.f1_score(y_all, all_predictions, average="macro", zero_division=0))
print("Accuracy - normalized:", sklearn.metrics.accuracy_score(y_all, all_predictions, normalize=True))  # fraction of correctly classified samples
print("Accuracy - unnormalized:", sklearn.metrics.accuracy_score(y_all, all_predictions, normalize=False))  # number of correctly classified samples

Precision - macro: 0.5549545231562777
Recall - macro: 0.4357427346662331
F1 Score - macro: 0.43723000467667783
Accuracy - normalized: 0.9929530514627141
Accuracy - unnormalized: 904049


In [32]:
print("Total samples:", X_all.shape[0])

Total samples: 910465


#### Evaluate: Each Label

There are more predictions than unique tokens, because with multilabel classification, one token can have multiple predicted tags.

In [22]:
exp_df = all_data.explode(["tag"])
exp_df = exp_df.rename(columns={"tag":"expected_tag"})
exp_df.head()

,sentence_id,token_id,token,pos,expected_tag
0,2,16,Scope,NN,O
1,2,17,and,CC,O
2,2,18,Contents,NNS,O
3,2,19,:,:,O
4,2,20,Sermons,NNS,O


In [23]:
print(exp_df.shape[0], len(exp_df.token_id.unique()))

915302 604541


In [24]:
exp_pred_df = pd.merge(
    left=exp_df, 
    right=pred_df.loc[pred_df.predicted_tag != "O"], # only include the predictions of Linguistic labels
    how="outer",
    left_on=["sentence_id", "token_id", "token", "pos", "expected_tag"],
    right_on=["sentence_id", "token_id", "token", "pos", "predicted_tag"],
    suffixes=["", "_pred"],
    indicator=True
)
exp_pred_df.shape

(922577, 7)

Record the agreement type for each row, ignoring rows with `'O'` and `NaN` value pairs (the `true negative` agreement type, which doesn't go into the precision, recall, or F1 score calculations).

In [25]:
exp_col = "expected_tag"
pred_col = "predicted_tag"
no_tag_value = "O"
# Find true negatives based on the expected and predicted tags
sub_exp_pred_df = exp_pred_df.loc[exp_pred_df[exp_col] == no_tag_value]
sub_exp_pred_df = sub_exp_pred_df.loc[sub_exp_pred_df[pred_col].isna()]
# sub_exp_pred_df.replace(to_replace="left_only", value="true negative", inplace=True)
tn_tokens = list(sub_exp_pred_df["token_id"])

# Record false negatives, false positives, and true positives based on the merge values
eval_df = exp_pred_df.loc[~exp_pred_df["token_id"].isin(tn_tokens)]
eval_df = eval_df.replace(to_replace="left_only", value="false negative")
eval_df = eval_df.replace(to_replace="right_only", value="false positive")
eval_df = eval_df.replace(to_replace="both", value="true positive")
eval_df = eval_df.sort_index()
eval_df.head()

,sentence_id,token_id,token,pos,expected_tag,predicted_tag,_merge
129,19,533,he,PRP,B-Gendered-Pronoun,B-Gendered-Pronoun,true positive
135,19,539,he,PRP,B-Gendered-Pronoun,B-Gendered-Pronoun,true positive
261,37,960,he,PRP,B-Gendered-Pronoun,B-Gendered-Pronoun,true positive
274,37,973,he,PRP,B-Gendered-Pronoun,B-Gendered-Pronoun,true positive
292,39,1002,his,PRP$,B-Gendered-Pronoun,B-Gendered-Pronoun,true positive


In [26]:
eval_df.shape

(9733, 7)

Save the data:

In [27]:
eval_df.to_csv(config.experiment1_agmt_path+"cc-{a}_ling_baseline_fastText{d}_evaluation_ALLDATA.csv".format(a=a,d=d))

##### Strict Agreement

Calculate the true positives, false positives, false negatives, precision, recall, and F1 metrics for all tags and each tag individually.

In [28]:
agmt_stats = utils.getAgreementStatsForAllTags(eval_df, "_merge", "token_id", "tag(s)", y_dev, predictions)
for label_tag in ling_label_subset:
    label_agmt_stats = utils.getScoresByTags(eval_df, "_merge", [label_tag])
    agmt_stats = pd.concat([agmt_stats, label_agmt_stats])
agmt_stats

,tag(s),false negative,false positive,true positive,precision,recall,f1
0,all,1283,21,8429,0.462383,0.422989,0.422518
0,B-Generalization,322,11,788,0.986233,0.709910,0.825563
0,I-Generalization,246,3,2,0.400000,0.008065,0.015810
0,B-Gendered-Role,241,2,2266,0.999118,0.903869,0.949110
0,I-Gendered-Role,385,0,0,0.000000,0.000000,0.000000
0,B-Gendered-Pronoun,30,5,13802,0.999638,0.997831,0.998734
0,I-Gendered-Pronoun,59,0,0,0.000000,0.000000,0.000000


Save the data:

In [29]:
agmt_stats.to_csv(config.experiment1_agmt_path+"cc-{a}_baseline_fastText{d}_ling_strict_agmt_ALLDATA.csv".format(a=a,d=d))

##### Annotation-level Agreement

Join the manual annotations' offsets to the evaluation data:

In [33]:
annot_df = pd.read_csv(config.agg_path+"aggregated_final.csv")#, usecols=["description_id","agg_ann_id", "ann_offsets"])
# Get only the Linguistic annotations
annot_df = annot_df.loc[annot_df.category == "Linguistic"]
annot_df = annot_df[["agg_ann_id", "ann_offsets", "label"]]
annot_df = annot_df.rename(columns={"agg_ann_id":"ann_id"})
# annot_df.head()

In [34]:
dev_token_ids = list(dev_data.token_id.unique())
ling_dev_subset = ling_dev.loc[ling_dev.token_id.isin(dev_token_ids)]

In [35]:
to_add = ling_dev_subset[["ann_id", "token_id", "token_offsets"]]
# Only include annotations with Linguistic labels
to_add = to_add.loc[to_add.ann_id.isin(list(annot_df.ann_id))]
eval_df_joined = eval_df.join(to_add.set_index("token_id"), on="token_id", how="outer")
# Join on the left, as there will be annotations from outside the devtest set in annot_df
print(eval_df_joined.shape)
eval_df_joined = eval_df_joined.join(annot_df.set_index("ann_id"), on="ann_id", how="left")
print(eval_df_joined.shape)  # Looks good!  Same as before join.
eval_df_joined = eval_df_joined.rename(columns={"label":"expected_label"})
# eval_df_joined.head()

(11834, 9)
(11834, 11)


Replace the predicted tags with their corresponding labels:

In [36]:
eval_df_joined.expected_label = eval_df_joined.expected_label.fillna("no_label")
eval_df_joined.expected_tag = eval_df_joined.expected_tag.fillna("no_label")
eval_df_joined.predicted_tag = eval_df_joined.predicted_tag.fillna("no_label")
# eval_df_joined.predicted_tag.value_counts()

In [37]:
predicted_labels = list(eval_df_joined.predicted_tag)
predicted_labels = [tag[2:] if tag != "no_label" else tag for tag in predicted_labels]
eval_df_joined.insert(len(eval_df_joined.columns), "predicted_label", predicted_labels)
# eval_df_joined.head()

In [38]:
cols_to_keep = ["sentence_id", "token_id", "token", "expected_label", "predicted_label", "_merge", "token_offsets", "ann_offsets", "ann_id"]
eval_by_ann = utils.implodeDataFrame(eval_df_joined[cols_to_keep], ["ann_id", "ann_offsets"]).reset_index()
exp_labels = list(eval_by_ann["expected_label"])
exp_labels = [labels[0] for labels in exp_labels]
eval_by_ann["expected_label"] = exp_labels
# eval_by_ann.head()

In [39]:
assert eval_by_ann.loc[eval_by_ann.expected_label == "no_label"].shape[0] == 0
assert eval_by_ann.loc[eval_by_ann.expected_label.isna()].shape[0] == 0

Every row should have an annotation label (a Linguistic label in `expected_label`).

In [40]:
ann_agmts = []
token_agmts = (eval_by_ann["_merge"])
for agmts in token_agmts:
    if "true positive" in agmts:
        ann_agmt = "true positive"
    elif "false positive" in agmts:
        ann_agmt = "false positive"
    else:
        ann_agmt = "false negative"
    ann_agmts += [ann_agmt]
assert len(ann_agmts) == eval_by_ann.shape[0]
eval_by_ann.insert(len(eval_by_ann.columns), "annotation_agreement", ann_agmts)
# eval_by_ann.head()

Save the data:

In [41]:
eval_by_ann[["ann_id", "ann_offsets", "token_id", "expected_label", "predicted_label", "annotation_agreement"]].to_csv(
    config.experiment1_agmt_path+"cc-{a}_baseline_fastText{d}_ling_annot_evaluation_ALLDATA.csv".format(a=a,d=d)
)

Calculate annotation agreement metrics for each label:

In [42]:
annot_agmt = pd.DataFrame.from_dict({
        "label":[], "false negative":[], "false positive":[],
         "true positive":[], "precision":[], "recall":[], "f1":[]
    })

In [43]:
labels = ling_label_tags.keys()
for label in labels:
    agmt_df = eval_by_ann.loc[eval_by_ann.expected_label == label]
    tp = agmt_df.loc[agmt_df.annotation_agreement == "true positive"].shape[0]
    fp = agmt_df.loc[agmt_df.annotation_agreement == "false positive"].shape[0]
    fn = agmt_df.loc[agmt_df.annotation_agreement == "false negative"].shape[0]
    prec, rec, f1 = utils.precisionRecallF1(tp, fp, fn)
    label_agmt = pd.DataFrame.from_dict({
            "label":[label], "false negative":[fn], "false positive":[fp],
             "true positive":[tp], "precision":[prec], "recall":[rec], "f1":[f1]
        })
    annot_agmt = pd.concat([annot_agmt, label_agmt])
annot_agmt

,label,false negative,false positive,true positive,precision,recall,f1
0,Gendered-Pronoun,131.0,0.0,1401.0,1.000000,0.914491,0.955336
0,Gendered-Role,976.0,4.0,230.0,0.982906,0.190713,0.319444
0,Generalization,408.0,3.0,124.0,0.976378,0.233083,0.376328


Save the scores:

In [44]:
# eval_by_ann.loc[eval_by_ann.expected_label == "Gendered-Role"]["annotation_agreement"].value_counts()     # Looks good
# eval_by_ann.loc[eval_by_ann.expected_label == "Gendered-Pronoun"]["annotation_agreement"].value_counts()  # Looks good
# eval_by_ann.loc[eval_by_ann.expected_label == "Generalization"]["annotation_agreement"].value_counts()    # Looks good
annot_agmt.to_csv(config.experiment1_agmt_path+"cc-{a}_baseline_fastText{d}_ling_annot_agmt_ALLDATA.csv".format(a=a,d=d))

##### Loose Agreement

Calculate precision, recall, and F1 score at the token level for each label, where a correct prediction is a prediction with the correct annotation label (not necessarily the correct IOB tag).

Create a copy of the evaluation DataFrame where tags are replaced by label names:

In [45]:
a = "rf"
eval_df = pd.read_csv(config.experiment1_agmt_path+"cc-{a}_ling_baseline_fastText{d}_evaluation_ALLDATA.csv".format(a=a,d=d), index_col=0)
loose_eval_df = eval_df.copy()
for label,tags in ling_label_tags.items():
    for tag in tags:
        loose_eval_df["expected_tag"] = loose_eval_df["expected_tag"].replace(to_replace=tag, value=label)
        loose_eval_df["predicted_tag"] = loose_eval_df["predicted_tag"].replace(to_replace=tag, value=label)
# loose_eval_df.head()

In [46]:
loose_eval_df.loc[loose_eval_df.predicted_tag.isna()].shape

(1283, 7)

In [52]:
loose_eval_df = loose_eval_df.fillna("O")
loose_eval_df = loose_eval_df.drop(columns=["_merge"])
loose_eval_df = utils.compareExpectedPredicted(loose_eval_df, "_merge", "O")
# loose_eval_df.head()

In [48]:
loose_eval_df.to_csv(config.experiment1_agmt_path+"cc-{a}_ling_baseline_fastText{d}_evaluation_loose_ALLDATA.csv".format(a=a,d=d))

In [49]:
loose_agmt = pd.DataFrame.from_dict({
        "tag(s)":[], "false negative":[], "false positive":[], "true negative":[], 
         "true positive":[], "precision":[], "recall":[], "f1":[]
    })

In [50]:
for label,tags in ling_label_tags.items():
    labels_agmt_stats = utils.getScoresByTags(loose_eval_df, "_merge", [label])
    loose_agmt = pd.concat([loose_agmt, labels_agmt_stats])
loose_agmt

,tag(s),false negative,false positive,true negative,true positive,precision,recall,f1
0,Gendered-Pronoun,89.0,0.0,NaN,13802.0,1.0,0.993593,0.996786
0,Gendered-Role,626.0,0.0,NaN,2266.0,1.0,0.783541,0.878635
0,Generalization,568.0,0.0,NaN,790.0,1.0,0.581738,0.735568


Save the data:

In [51]:
loose_agmt.to_csv(config.experiment1_agmt_path+"cc-{a}_baseline_fastText{d}_ling_loose_agmt_ALLDATA.csv".format(a=a,d=d))